In [18]:
# Import the required packeages
import pandas as pd
import urllib3
urllib3.disable_warnings()
from pandas.io.json import json_normalize
from typing import List, Dict
import sys
import os
import numpy as np
import concurrent.futures
import json
import itertools
from aiohttp import ClientSession
import asyncio

In [2]:
# Setting the path
module_path = os.path.abspath(os.path.join("../src/pyprediktormapclient/"))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/home/meena/dev/pyPrediktorMapClient/src/pyprediktormapclient


In [3]:
# Import model index functions
from model_index import ModelIndex

# Import OPC UA functions
from opc_ua import OPC_UA

In [4]:
# Connection to the servers
model_index_url = "http://10.241.68.86:7001/v1/"
opc_url = "http://10.241.68.86:13371/"

# Model index API
mdx = ModelIndex(url=model_index_url)

# OPC UA API
opc = OPC_UA(url=opc_url)

In [5]:
# Input parameters for value data
server_url = "opc.tcp://10.241.80.4:4872"
# Parameters for aggregate historical data
start_time = "2022-07-05T07:55:14.544000Z" # 5th July aggregated data 
end_time = "2022-07-06T07:55:14.544000Z"
pro_interval = 600000 # 10 minutes processing time
agg_name = "Average"

pro_interval1 = 3600000 # 1 hour processing time
end_time1 = "2022-07-9T07:55:14.544000Z" # 5 days
end_time2 = "2022-07-14T07:55:14.544000Z" # 10 days
end_time3 = "2022-08-05T07:55:14.544000Z" # 30 days

In [6]:
# Strings set type data of the site BR-AP
strings = mdx.get_object_descendants("StringSetType", ['4:1:SSO.EG-AS'], "PV_Assets")

In [7]:
# Node ids of the variables (Nodes)
vars_node_ids = mdx.get_vars_node_ids(strings) 

In [8]:
''', 'ModuleNominalDCPower', 'DCCurrent', 'Width'
'''

", 'ModuleNominalDCPower', 'DCCurrent', 'Width'\n"

In [9]:
variable_list = ['Height']

In [10]:
'''var_ids = [x for x in vars_node_ids if (x.split(".")[-1]) in variable_list]

for s in strings:
    s['Vars'] = [x for x in s['Vars'] if x['DisplayName'] in variable_list]

var_node_ids = [x for x in strings_df['VariableId'].to_list() if (x.split(".")[-1]) in variable_list]

strings_var_ids = mdx.read_var_ids(strings, variable_list)'''

'var_ids = [x for x in vars_node_ids if (x.split(".")[-1]) in variable_list]\n\nfor s in strings:\n    s[\'Vars\'] = [x for x in s[\'Vars\'] if x[\'DisplayName\'] in variable_list]\n\nvar_node_ids = [x for x in strings_df[\'VariableId\'].to_list() if (x.split(".")[-1]) in variable_list]\n\nstrings_var_ids = mdx.read_var_ids(strings, variable_list)'

In [11]:
# Convert the json data into dataframe
strings_df = mdx.expand_props_vars(strings)

In [12]:
strings_df1 = strings_df[strings_df['Variable'].isin(variable_list)].reset_index(drop=True)

In [13]:
livValues = opc.get_live_values(server_url, variable_list, vars_node_ids)

In [14]:
livValues_df = opc.get_live_values_dataframe(server_url, variable_list, strings_df)

In [15]:
livValues_df

,ObjectId,DescendantId,DescendantName,DescendantType,ObjectName,Variable,VariableId,ColPosition,GPSLatitude,GPSLongitude,...,ModuleMaterialCategory,NumberOfModules,NumberOfStrings,RelativeXPosition,RelativeYPosition,RowPosition,SourceTimestamp,ServerTimestamp,Value.Type,Value.Body
0,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH01,EG-AS-TS10-I19-SM01-CH01,StringSetType,EG-AS,Height,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH01.Paramet...,1,24.4373642851457,32.7319184191594,...,CSI,0,2.0,1616.00637020002,469.894599999767,37,2022-04-27T10:17:46.346Z,2022-08-03T09:14:06.3785587Z,10,2
1,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH02,EG-AS-TS10-I19-SM01-CH02,StringSetType,EG-AS,Height,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH02.Paramet...,2,24.437646408359,32.7319184191594,...,CSI,0,2.0,1616.00637020002,501.080499999691,37,2022-04-27T10:17:46.346Z,2022-08-03T09:14:06.3785593Z,10,2
2,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH03,EG-AS-TS10-I19-SM01-CH03,StringSetType,EG-AS,Height,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH03.Paramet...,3,24.437646408359,32.7320113121203,...,CSI,0,2.0,1624.70607019993,501.080499999691,37,2022-04-27T10:17:46.346Z,2022-08-03T09:14:06.377669Z,10,2
3,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH04,EG-AS-TS10-I19-SM01-CH04,StringSetType,EG-AS,Height,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH04.Paramet...,4,24.4373642851457,32.7320113121203,...,CSI,0,2.0,1624.70607019993,469.894599999767,37,2022-04-27T10:17:46.346Z,2022-08-03T09:14:06.3785411Z,10,2
4,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH05,EG-AS-TS10-I19-SM01-CH05,StringSetType,EG-AS,Height,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH05.Paramet...,5,24.4373639563054,32.7321042050812,...,CSI,0,2.0,1633.40577019995,469.858249999583,37,2022-04-27T10:17:46.346Z,2022-08-03T09:14:06.3785608Z,10,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH06,EG-AS-TS05-I10-SM13-CH06,StringSetType,EG-AS,Height,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH06.Paramet...,126,24.4315926156143,32.734890993908,...,CSI,0,2.0,1894.39677019995,-168.105750000105,19,2022-04-27T10:17:46.438Z,2022-08-03T09:14:06.3665212Z,10,2
2934,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH07,EG-AS-TS05-I10-SM13-CH07,StringSetType,EG-AS,Height,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH07.Paramet...,127,24.4315926156143,32.7349838868689,...,CSI,0,2.0,1903.09647019997,-168.105750000105,19,2022-04-27T10:17:46.438Z,2022-08-03T09:14:06.366549Z,10,2
2935,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH08,EG-AS-TS05-I10-SM13-CH08,StringSetType,EG-AS,Height,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH08.Paramet...,128,24.4318727259815,32.7349838868689,...,CSI,0,2.0,1903.09647019997,-137.142350000329,19,2022-04-27T10:17:46.438Z,2022-08-03T09:14:06.3665292Z,10,2
2936,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH09,EG-AS-TS05-I10-SM13-CH09,StringSetType,EG-AS,Height,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH09.Paramet...,129,24.4318727259815,32.7350767798298,...,CSI,0,2.0,1911.7961702,-137.142350000329,19,2022-04-27T10:17:46.438Z,2022-08-03T09:14:06.3665504Z,10,2


In [15]:
df = pd.json_normalize(opc.get_live_values(server_url, variable_list, strings_df['VariableId'].to_list())[0]['Values']).reset_index(drop=True)

In [16]:
df

,SourceTimestamp,ServerTimestamp,Value.Type,Value.Body
0,2022-04-27T10:17:46.346Z,2022-08-03T09:14:06.3785587Z,10,2
1,2022-04-27T10:17:46.346Z,2022-08-03T09:14:06.3785593Z,10,2
2,2022-04-27T10:17:46.346Z,2022-08-03T09:14:06.377669Z,10,2
3,2022-04-27T10:17:46.346Z,2022-08-03T09:14:06.3785411Z,10,2
4,2022-04-27T10:17:46.346Z,2022-08-03T09:14:06.3785608Z,10,2
...,...,...,...,...
2933,2022-04-27T10:17:46.438Z,2022-08-03T09:14:06.3665212Z,10,2
2934,2022-04-27T10:17:46.438Z,2022-08-03T09:14:06.366549Z,10,2
2935,2022-04-27T10:17:46.438Z,2022-08-03T09:14:06.3665292Z,10,2
2936,2022-04-27T10:17:46.438Z,2022-08-03T09:14:06.3665504Z,10,2


In [22]:
final_df = pd.concat([strings_df1, df], axis=1)

In [23]:
final_df

,ObjectId,DescendantId,DescendantName,DescendantType,ObjectName,Variable,VariableId,ColPosition,GPSLatitude,GPSLongitude,...,ModuleMaterialCategory,NumberOfModules,NumberOfStrings,RelativeXPosition,RelativeYPosition,RowPosition,SourceTimestamp,ServerTimestamp,Value.Type,Value.Body
0,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH01,EG-AS-TS10-I19-SM01-CH01,StringSetType,EG-AS,Height,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH01.Paramet...,1,24.4373642851457,32.7319184191594,...,CSI,0,2.0,1616.00637020002,469.894599999767,37,2022-04-27T10:17:46.346Z,2022-08-03T09:14:06.3785587Z,10,2
1,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH02,EG-AS-TS10-I19-SM01-CH02,StringSetType,EG-AS,Height,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH02.Paramet...,2,24.437646408359,32.7319184191594,...,CSI,0,2.0,1616.00637020002,501.080499999691,37,2022-04-27T10:17:46.346Z,2022-08-03T09:14:06.3785593Z,10,2
2,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH03,EG-AS-TS10-I19-SM01-CH03,StringSetType,EG-AS,Height,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH03.Paramet...,3,24.437646408359,32.7320113121203,...,CSI,0,2.0,1624.70607019993,501.080499999691,37,2022-04-27T10:17:46.346Z,2022-08-03T09:14:06.377669Z,10,2
3,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH04,EG-AS-TS10-I19-SM01-CH04,StringSetType,EG-AS,Height,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH04.Paramet...,4,24.4373642851457,32.7320113121203,...,CSI,0,2.0,1624.70607019993,469.894599999767,37,2022-04-27T10:17:46.346Z,2022-08-03T09:14:06.3785411Z,10,2
4,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH05,EG-AS-TS10-I19-SM01-CH05,StringSetType,EG-AS,Height,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH05.Paramet...,5,24.4373639563054,32.7321042050812,...,CSI,0,2.0,1633.40577019995,469.858249999583,37,2022-04-27T10:17:46.346Z,2022-08-03T09:14:06.3785608Z,10,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH06,EG-AS-TS05-I10-SM13-CH06,StringSetType,EG-AS,Height,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH06.Paramet...,126,24.4315926156143,32.734890993908,...,CSI,0,2.0,1894.39677019995,-168.105750000105,19,2022-04-27T10:17:46.438Z,2022-08-03T09:14:06.3665212Z,10,2
2934,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH07,EG-AS-TS05-I10-SM13-CH07,StringSetType,EG-AS,Height,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH07.Paramet...,127,24.4315926156143,32.7349838868689,...,CSI,0,2.0,1903.09647019997,-168.105750000105,19,2022-04-27T10:17:46.438Z,2022-08-03T09:14:06.366549Z,10,2
2935,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH08,EG-AS-TS05-I10-SM13-CH08,StringSetType,EG-AS,Height,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH08.Paramet...,128,24.4318727259815,32.7349838868689,...,CSI,0,2.0,1903.09647019997,-137.142350000329,19,2022-04-27T10:17:46.438Z,2022-08-03T09:14:06.3665292Z,10,2
2936,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH09,EG-AS-TS05-I10-SM13-CH09,StringSetType,EG-AS,Height,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH09.Paramet...,129,24.4318727259815,32.7350767798298,...,CSI,0,2.0,1911.7961702,-137.142350000329,19,2022-04-27T10:17:46.438Z,2022-08-03T09:14:06.3665504Z,10,2


In [ ]:
liv_values = pd.json_normalize(opc.get_live_values(server_url, strings_var_ids['VariableId'].to_list())[0]['Values'])

In [ ]:
liv_values

In [ ]:
df_new = pd.concat([strings_df, liv_values], axis=1)

In [ ]:
len(df_new)

In [ ]:
liv_values_df = opc.get_live_values_dataframe(server_url, strings_df)

In [ ]:
len(liv_values_df)

In [ ]:
# Object ids of strings set type objects
string_objects_ids = [x['DescendantId'] for x in strings]

In [ ]:
# Convert the json data into dataframe
#strings_df = mdx.expand_props_vars(strings)
#strings_df

In [ ]:
#strings_df1 = strings_df[:5]
#strings_df1

In [ ]:
# Node ids of the variables (Nodes)
vars_node_ids = mdx.get_vars_node_ids(strings) 

In [ ]:
#vars_node_ids1 = vars_node_ids[:20000]

In [24]:
vars_node_ids1 = vars_node_ids[:5]

In [ ]:
#large_agg_hist_month = opc.get_agg_hist_value_chunks(server_url, start_time, end_time3, pro_interval, agg_name, vars_node_ids, 100000, 5000, 4)

In [25]:
chunk_size = 100000
batch_size = 500
read_value_ids = [opc.create_readvalueids_dict(x,agg_name) for x in vars_node_ids1]
# Lenght of time series
n_datapoints = (pd.to_datetime(end_time) - pd.to_datetime(start_time)).total_seconds()*1000/pro_interval
# Number of required splits 
n_time_splits = int(np.ceil(n_datapoints/chunk_size))
# Ids chunks
id_chunk_list = list(opc.chunk_ids(read_value_ids, batch_size))
# Get datetime chunks
start_end_list = opc.chunk_datetimes(start_time,end_time, n_time_splits)
body_elements = list(itertools.product(start_end_list,id_chunk_list))
# Create body chunks
body_list = []
for x in body_elements:
    start_time_new = x[0][0]
    end_time_new = x[0][1]
    ids = x[1]
    body = json.dumps({
            "Connection": {
                "Url": server_url,
                "AuthenticationType": 1
            },
            "StartTime": start_time_new,
            "EndTime": end_time_new,
            "ProcessingInterval": pro_interval, 
            "ReadValueIds": ids
        
        })
    body_list.append(body)
#
endpoint = 'values/historicalaggregated'

In [55]:
body_chunks_list = list(opc.chunk_ids(body_list[:1], 2))
# Request chunkwise data
timeout= 1000000000000000000000000000000
session = ClientSession()
results = await opc.http_get_with_aiohttp(session, endpoint,body_chunks_list[0][0],timeout)
await session.close()

True

In [65]:
def filter_read_results(x):
    y = {}
    y['NodeId'] = x['NodeId']['Id']
    xvalues = x['DataValues']
    for v in xvalues:
        v['Value'] = v['Value']['Body']
    y['DataValues'] = xvalues
    return y

def filter_json_response(response_json):
    response_json.pop("ServerNamespaces", None)
    response_json.pop("Success", None)
    response_json['HistoryReadResults'] = [self.filter_read_results(x) for x in response_json['HistoryReadResults']]
    return response_json


In [66]:
results.pop("ServerNamespaces", None)
results.pop("Success", None)
results['HistoryReadResults'] = [filter_read_results(x) for x in results['HistoryReadResults']]

In [67]:
results

{'HistoryReadResults': [{'NodeId': 'SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH01.Parameters.Height',
   'DataValues': [{'Value': 0,
     'StatusCode': {'Code': 2157641728, 'Symbol': 'BadNoData'},
     'SourceTimestamp': '2022-07-05T07:55:14.544Z'},
    {'Value': 0,
     'StatusCode': {'Code': 2157641728, 'Symbol': 'BadNoData'},
     'SourceTimestamp': '2022-07-05T08:05:14.544Z'},
    {'Value': 0,
     'StatusCode': {'Code': 2157641728, 'Symbol': 'BadNoData'},
     'SourceTimestamp': '2022-07-05T08:15:14.544Z'},
    {'Value': 0,
     'StatusCode': {'Code': 2157641728, 'Symbol': 'BadNoData'},
     'SourceTimestamp': '2022-07-05T08:25:14.544Z'},
    {'Value': 0,
     'StatusCode': {'Code': 2157641728, 'Symbol': 'BadNoData'},
     'SourceTimestamp': '2022-07-05T08:35:14.544Z'},
    {'Value': 0,
     'StatusCode': {'Code': 2157641728, 'Symbol': 'BadNoData'},
     'SourceTimestamp': '2022-07-05T08:45:14.544Z'},
    {'Value': 0,
     'StatusCode': {'Code': 2157641728, 'Symbol': 'BadNoData'},
     'Sour

In [ ]:
for r in results['HistoryReadResults']:
    r['NodeId'] = [x for x in r['NodeId'] if x['DisplayName'] in variable_list]

In [53]:
oldDict = results['HistoryReadResults']

In [54]:
new_dict = {Id: oldDict['NodeId'] for Id in oldDict}

TypeError: list indices must be integers or slices, not str

In [ ]:
dictA = {'Sun': '2 PM', "Tue": '5 PM', 'Wed': '3 PM', 'Fri': '9 PM'}

res = {key: dictA[key] for key in dictA.keys()
& {'Fri', 'Sun'}}

In [46]:
dict = { NodeId:  results['NodeId'] for NodeId in your_keys }

KeyError: 'NodeId'

In [45]:
dict

{'NodeId': [{'NodeId': {'IdType': 1,
    'Id': 'SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH01.Parameters.Height',
    'Namespace': 4},
   'StatusCode': {'Code': 0, 'Symbol': 'Good'},
   'DataValues': [{'Value': {'Type': 11, 'Body': 0},
     'StatusCode': {'Code': 2157641728, 'Symbol': 'BadNoData'},
     'SourceTimestamp': '2022-07-05T07:55:14.544Z'},
    {'Value': {'Type': 11, 'Body': 0},
     'StatusCode': {'Code': 2157641728, 'Symbol': 'BadNoData'},
     'SourceTimestamp': '2022-07-05T08:05:14.544Z'},
    {'Value': {'Type': 11, 'Body': 0},
     'StatusCode': {'Code': 2157641728, 'Symbol': 'BadNoData'},
     'SourceTimestamp': '2022-07-05T08:15:14.544Z'},
    {'Value': {'Type': 11, 'Body': 0},
     'StatusCode': {'Code': 2157641728, 'Symbol': 'BadNoData'},
     'SourceTimestamp': '2022-07-05T08:25:14.544Z'},
    {'Value': {'Type': 11, 'Body': 0},
     'StatusCode': {'Code': 2157641728, 'Symbol': 'BadNoData'},
     'SourceTimestamp': '2022-07-05T08:35:14.544Z'},
    {'Value': {'Type': 11, 'Body':

In [40]:
for r in results['HistoryReadResults']:
    r['NodeId'] = [x for x in r['Id']]

KeyError: 'Id'

In [37]:
pd.DataFrame(results['HistoryReadResults'])

,NodeId,StatusCode,DataValues
0,"[S, S, O, ., E, G, -, A, S, ., S, 1, ., Z, 1, ...","{'Code': 0, 'Symbol': 'Good'}","[{'Value': {'Type': 11, 'Body': 0}, 'StatusCod..."
1,"[S, S, O, ., E, G, -, A, S, ., S, 1, ., Z, 1, ...","{'Code': 0, 'Symbol': 'Good'}","[{'Value': {'Type': 11, 'Body': 0}, 'StatusCod..."
2,"[S, S, O, ., E, G, -, A, S, ., S, 1, ., Z, 1, ...","{'Code': 0, 'Symbol': 'Good'}","[{'Value': {'Type': 11, 'Body': 0}, 'StatusCod..."
3,"[S, S, O, ., E, G, -, A, S, ., S, 1, ., Z, 1, ...","{'Code': 0, 'Symbol': 'Good'}","[{'Value': {'Type': 11, 'Body': 0}, 'StatusCod..."
4,"[S, S, O, ., E, G, -, A, S, ., S, 1, ., Z, 1, ...","{'Code': 0, 'Symbol': 'Good'}","[{'Value': {'Type': 11, 'Body': 0}, 'StatusCod..."


In [28]:
results['HistoryReadResults']

[{'NodeId': {'IdType': 1,
   'Id': 'SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH01.Parameters.Height',
   'Namespace': 4},
  'StatusCode': {'Code': 0, 'Symbol': 'Good'},
  'DataValues': [{'Value': {'Type': 11, 'Body': 0},
    'StatusCode': {'Code': 2157641728, 'Symbol': 'BadNoData'},
    'SourceTimestamp': '2022-07-05T07:55:14.544Z'},
   {'Value': {'Type': 11, 'Body': 0},
    'StatusCode': {'Code': 2157641728, 'Symbol': 'BadNoData'},
    'SourceTimestamp': '2022-07-05T08:05:14.544Z'},
   {'Value': {'Type': 11, 'Body': 0},
    'StatusCode': {'Code': 2157641728, 'Symbol': 'BadNoData'},
    'SourceTimestamp': '2022-07-05T08:15:14.544Z'},
   {'Value': {'Type': 11, 'Body': 0},
    'StatusCode': {'Code': 2157641728, 'Symbol': 'BadNoData'},
    'SourceTimestamp': '2022-07-05T08:25:14.544Z'},
   {'Value': {'Type': 11, 'Body': 0},
    'StatusCode': {'Code': 2157641728, 'Symbol': 'BadNoData'},
    'SourceTimestamp': '2022-07-05T08:35:14.544Z'},
   {'Value': {'Type': 11, 'Body': 0},
    'StatusCode': {'Code'

In [ ]:
with open('test.json', 'w') as f:
    json.dump(results, f)

In [ ]:
df_results = pd.DataFrame(results)
df_results

In [ ]:
results_list = []
for res in results:
    for x in res['HistoryReadResults']:
        results_list.append(x)

len(results_list)

In [ ]:
df_results = pd.DataFrame(results_list)
df_results

In [ ]:
df_results.to_parquet('test.parquet')

In [ ]:
df_results1 = pd.read_parquet('test.parquet')
df_results1

In [ ]:
df_results1['DataValues'][0]

In [ ]:
# Strings set type data of the site BR-AP
strings = mdx.get_object_descendants("StringSetType", ['5:1:SSO.EG-DA'], "PV_Assets")

# Object ids of strings set type objects
string_objects_ids = [x['DescendantId'] for x in strings]

# Node ids of the variables (Nodes)
vars_node_ids = mdx.get_vars_node_ids(strings) 

vars_node_ids1 = vars_node_ids[:10]

OPC value function

In [ ]:
test_results = opc.get_agg_hist_values(server_url, start_time, end_time, pro_interval, agg_name, vars_node_ids1)

In [ ]:
test_results1 = pd.DataFrame(test_results)

In [ ]:
len(test_results1['HistoryReadResults'])

In [ ]:
test_results1['HistoryReadResults'][0]

In [ ]:
hist = pd.DataFrame(test_results1['HistoryReadResults'])
hist

In [ ]:
hist[['NodeId','StatusCode', 'SourceTimestamp']] = hist['HistoryReadResults'].apply(pd.Series)
hist

In [ ]:
test1 = pd.json_normalize(test_results1['HistoryReadResults'][0])
test1

In [ ]:
data_value1 = test1['DataValues'][0]
data_value1

In [ ]:
data_value2 = pd.DataFrame(data_value1)
data_value2

In [ ]:
data_value3 = pd.json_normalize(data_value1)
data_value3

In [ ]:
df1 = df_results1.explode('DataValues').reset_index(drop=True)
pd.json_normalize(df1)

In [ ]:
df1['DataValues'].keys()

In [ ]:
# 1 day parquet
session = ClientSession()
await opc.get_agg_hist_value_chunks_parallel(session,server_url, start_time, end_time, pro_interval, agg_name, vars_node_ids, 100000,1000,10,1000000000000000000000000000000)
await session.close()

In [ ]:
one_day_df = pd.read_parquet('data/')
one_day_df

Columns to keep in final dataframe.
Particular node ids for variables.
Rename of columns variables

In [ ]:
# 1 month
session = ClientSession()
await opc.get_agg_hist_value_chunks_new(session,server_url, start_time, end_time3, pro_interval, agg_name, vars_node_ids, 100000,200,5,1000000000000000000000000000000)
await session.close()

There are two more possibilities to improve the code. 1. Requesting only required columns by filtering the API request. 2. Saving data in parquet format instead of JSON. 

One thousands of node ids took 6 minutes to return with one month data. Therefor, approximately 1.45 lac node ids with month data will return in about 14 to 15 hours.

In [ ]:
session = ClientSession()
await opc.get_agg_hist_value_chunks_new(session,server_url, start_time, end_time1, pro_interval, agg_name, vars_node_ids, 100000,1000,10,10000000000000000000000000000)
await session.close()

5 days data with time chunk size 100000, id batch size 1000 and maximum workers 10 took around 2 and half hour.

In [ ]:
session = ClientSession()
await opc.get_agg_hist_value_chunks_new(session,server_url, start_time, end_time, pro_interval, agg_name, vars_node_ids, 100000,1000,10,10000000000000000000000000000)
await session.close()

One day data return in around 30-40 minutes. 

In [ ]:
session = ClientSession()
await opc.get_agg_hist_value_chunks_new(session,server_url, start_time, end_time, pro_interval, agg_name, vars_node_ids, 100000, 500,10,10000000000000000000000000000)
await session.close()

In [ ]:
session = ClientSession()
results = await asyncio.gather(*[opc.http_get_with_aiohttp(session, endpoint,body,10000000000000000000000000000) for body in body_list])
await session.close()

In [ ]:
len(results)

In [ ]:
results

In [ ]:
len([x for x in results if type(x)==dict])

In [ ]:
len(results[0]['HistoryReadResults'])

In [ ]:
len([x for x in results if x['HistoryReadResults'] is None])

In [ ]:
results[0]

In [ ]:
large_agg_hist2 = opc.get_agg_hist_value_chunks(server_url, start_time, end_time, pro_interval, agg_name, vars_node_ids, 100000, 1000, 4)

Around 145000 strings set node ids with 4 max workers and ids bach  size 1000 took about 24 minutes and 17 seconds.

In [ ]:
large_agg_hist5 = opc.get_agg_hist_value_chunks(server_url, start_time, end_time, pro_interval, agg_name, vars_node_ids, 100000, 5000, 4)

In [ ]:
large_agg_hist = opc.get_agg_hist_value_chunks(server_url, start_time, end_time, pro_interval, agg_name, vars_node_ids, 100000, 5000, 4)

Around 145000 strings set node ids with 4 max workers and ids bach  size 5000 took about 24 minutes and 57 seconds. Proccessing interval 10 minutes.

In [ ]:
large_agg_hist_2 = opc.get_agg_hist_value_chunks(server_url, start_time, end_time1, pro_interval1, agg_name, vars_node_ids, 100000, 5000, 4)

In [ ]:
len(large_agg_hist_2)

In [ ]:
d_value = opc.get_agg_hist_values_dataframe(server_url, start_time, end_time, pro_interval, agg_name, strings_df1)

In [ ]:
d_value.head()

In [ ]:
read_value_ids = [opc.create_readvalueids_dict(x,agg_name) for x in vars_node_ids]
# Lenght of time series
n_datapoints = (pd.to_datetime(end_time) - pd.to_datetime(start_time)).total_seconds()*1000/pro_interval

# Number of nodeids to read
n_nodeids = len(vars_node_ids)
total_datapoints = n_nodeids*n_datapoints

chunk_size = 100000
# Number of required splits 
n_time_splits = int(np.ceil(n_datapoints/chunk_size))

In [ ]:
def chunk_ids(ids_list, n):
    """Yield successive n-sized chunks from ids_list."""
    for i in range(0, len(ids_list), n):
        yield ids_list[i:i + n]

In [ ]:
id_batch_size = 1000
id_chunk_list = list(chunk_ids(read_value_ids, id_batch_size))

In [ ]:
# Get datetime chunks
start_end_list = opc.chunk_datetimes(start_time,end_time, n_time_splits)

# Body elements to pass in API request body
body_elements = list(itertools.product(start_end_list,id_chunk_list))

In [ ]:
# Create body chunks
body_list = []
for x in body_elements:
    start_time_new = x[0][0]
    end_time_new = x[0][1]
    ids = x[1]
    body = json.dumps({
            "Connection": {
                "Url": server_url,
                "AuthenticationType": 1
            },
            "StartTime": start_time_new,
            "EndTime": end_time_new,
            "ProcessingInterval": pro_interval, 
            "ReadValueIds": ids
        
        })
    body_list.append(body)

In [ ]:
len(body_list)

In [ ]:
data_0 = opc.request_historical_data(body_list[0])

In [ ]:
data_0

In [ ]:
max_workers1 = 2

In [ ]:
'''# Request chunkwise data
data_list = []
for i,body in enumerate(body_list[:2]):
    try:
        recieved_frame = opc.request_historical_data(body)
        print("Succeeded : "+str(i))
        data_list+=recieved_frame['HistoryReadResults']
    except:
        print("Failed : "+str(i))
    '''

In [ ]:
from concurrent.futures import as_completed

In [ ]:
# Request chunkwise data
data_list = []
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    future_data_map = {executor.submit(opc.request_historical_data, chunk):i for i,chunk in enumerate(body_list[:10])}
    for i,future_data in enumerate(concurrent.futures.as_completed(future_data_map)):
        try:
            print("Success: "+str(i))
            recieved_frame = future_data.result()
            data_list+=recieved_frame['HistoryReadResults']
        except:
            print("Failed: "+str(i))

In [ ]:
len(data_list)

In [ ]:
# Request chunkwise data
data_list = []
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    future_data_map = {executor.submit(opc.request_historical_data, chunk):i for i,chunk in enumerate(body_list[:10])}
    for i,future_data in enumerate(concurrent.futures.as_completed(future_data_map)):
        try:
            print("Success: "+str(i))
            recieved_frame = future_data.result()
            data_list+=recieved_frame['HistoryReadResults']
        except:
            print("Failed: "+str(i))

In [ ]:
len(data_0)
data_0.keys()
len(data_0['HistoryReadResults'])
data_0['HistoryReadResults'][0]
len(data_0['HistoryReadResults'][0]['DataValues'])

In [ ]:
max_workers = 4

In [ ]:
# Request chunkwise data
data_list = []
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    future_data_map = {executor.submit(opc.request_historical_data, chunk):i for i,chunk in enumerate(body_list)}
    for future_data in concurrent.futures.as_completed(future_data_map):
        try:
            recieved_frame = future_data.result()
        except:
            recieved_frame = []
        data_list.append(recieved_frame)

In [ ]:
# agg_hist_df = opc.get_agg_hist_values(server_url, start_time, end_time, pro_interval, agg_name, vars_node_ids)

In [ ]:
'''        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_data_map = {executor.submit(self.request_historical_data, chunk):i for i,chunk in enumerate(body_list)}
            for future_data in concurrent.futures.as_completed(future_data_map):
                try:
                    recieved_frame = future_data.result()
                except:
                    recieved_frame = []
                data_list.append(recieved_frame)'''

In [ ]:
'''
    def get_agg_hist_values_dataframe(self, server_url: str, start_time: str, end_time: str, pro_interval: int, agg_name: str, data_frame : pd.DataFrame) -> pd.DataFrame:
        """Make a dataframe of aggregated historical value data

        Args:
            server_url (str): server connection url
            start_time (str): start time of requested data
            end_time (str): end time of the requested data
            pro_interval (int): interval time of processing in milliseconds
            agg_name (str): Name of aggregation
            data_frame (pd.DataFrame): Pandas data frame with required columns such as node_ids

        Returns:
            pd.DataFrame: Dataframe of the requested historical values
        """
        # JSON normalization of aggregate historical values
        df = pd.json_normalize(self.get_agg_hist_values(server_url, start_time, end_time, pro_interval, agg_name, data_frame['VariableId'].to_list())['HistoryReadResults'])
        # Concating aggregated historical values to dataframe
        data_frame1 = pd.concat([data_frame,df], axis=1).drop(columns=['NodeId.IdType', 'NodeId.Id', 'NodeId.Namespace', 'StatusCode.Code', 'StatusCode.Symbol']) 
        # Exploding DataValues column
        df1 = data_frame1.explode('DataValues').reset_index(drop=True)
        # JSON normalization of DataValues
        df2 = pd.json_normalize(df1['DataValues'])
        # Concatenating dataframes
        data_frame2 = pd.concat([df1,df2], axis=1).drop(columns=["DataValues"])
        return data_frame2
        '''

Variable name Function

In [ ]:
json_data = mdx.get_objects_of_type('SiteType')

In [ ]:
json_file = mdx.expand_props_vars(json_data)

In [ ]:
var_list = list(pd.unique(json_file['Variable']))

In [ ]:
lst = ['Altitude']

In [ ]:
def read_var_ids(json_data : List[Dict], vars:list):
    json_df = mdx.expand_props_vars(json_data)
    dataframe = json_df[json_df['Variable'].isin(vars)]#['VariableId']
    return dataframe

In [ ]:
df_var = read_var_ids(json_data, lst)
len(df_var)

In [ ]:
val_live = pd.json_normalize(opc.get_live_values(server_url, df_var['VariableId'].to_list())[0]['Values'])

In [ ]:
len(val_live)

In [ ]:
new_df = pd.concat([df_var, val_live], axis=1)

In [ ]:
len(new_df)

In [ ]:
var = json_file[json_file['Variable'].isin(var_list)]['VariableId']

In [ ]:
json_df = pd.DataFrame(json_data)

In [ ]:
json_df['var_list'] = json_df['Vars'].apply(lambda l: [y['DisplayName'] for y in l])

In [ ]:
json_df = json_df.explode('Vars').reset_index(drop=True)

In [ ]:
json_df[['Variable','VariableId']] = json_df['Vars'].apply(pd.Series)
json_df = json_df.drop(columns=['Props','Vars','type','subtype'])

In [ ]:
json_df[json_df['Variable'].isin(['Altitude'])]#['VariableId']]

In [ ]:
# Creating a column with list of all the variables of Vars column
site_objects_df['Variables'] = site_objects_df['Vars'].apply(lambda l: [y['DisplayName'] for y in l])
Vars_list = site_objects_df.Variables

In [ ]:
non_vars_props_columns = [x for x in json_df.columns if x not in ['Vars','Props']]

In [ ]:
json_df1 = json_df.drop(columns=['Vars']).explode('Props').reset_index(drop=True)

def get_live_values_dataframe(self, server_url: str, var_node_ids: List[str], data_frame : pd.DataFrame) -> pd.DataFrame:
        """Make a dataframe of the live values from the server 

        Args:
            server_url (str): server connection url
            data_frame (pd.DataFrame): Pandas data frame with required columns

        Returns:
            pd.DataFrame: Dataframe of the requested live values
        """
        # JSON normalization of the live values
        df = pd.json_normalize(self.get_live_values(server_url, data_frame['VariableId'].to_list())[0]['Values'])
        # Concating both the dataframes
        final_df = pd.concat([df,data_frame], axis=1)
        return final_df

async def get_agg_hist_value_chunks_parallel(self,session: ClientSession, server_url: str, start_time: str, end_time: str, pro_interval: int, agg_name: str, node_ids: List[str], chunk_size=100000,batch_size=1000,max_workers=4,timeout: int = 10):
        """Function to make aiohttp based multithreaded API request to get aggregated historical data from OPC UA API

        Args:
            session (ClientSession): Session of one request
            server_url (str): server connection url
            start_time (str): start time of requested data
            end_time (str): end time of requested data
            pro_interval (int): interval time of processing in milliseconds
            agg_name (str): Name of aggregation
            node_ids (List[str]): node id(s)
            chunk_size (int, optional): Time chunk size. Defaults to 100000.
            batch_size (int, optional): size of each Id chunck. Defaults to 1000.
            max_workers (int, optional): maximum number of workers(CPU). Defaults to 10.
            timeout (int, optional): Timeout time of one session. Defaults to 10.
        """
        read_value_ids = [self.create_readvalueids_dict(x,agg_name) for x in node_ids]
        # Lenght of time series
        n_datapoints = (pd.to_datetime(end_time) - pd.to_datetime(start_time)).total_seconds()*1000/pro_interval
        # Number of required splits 
        n_time_splits = int(np.ceil(n_datapoints/chunk_size))
        # Ids chunks
        id_chunk_list = list(self.chunk_ids(read_value_ids, batch_size))
        # Get datetime chunks
        start_end_list = self.chunk_datetimes(start_time,end_time, n_time_splits)
        body_elements = list(itertools.product(start_end_list,id_chunk_list))
        # Create body chunks
        body_list = []
        for x in body_elements:
            start_time_new = x[0][0]
            end_time_new = x[0][1]
            ids = x[1]
            body = json.dumps({
                    "Connection": {
                        "Url": server_url,
                        "AuthenticationType": 1
                    },
                    "StartTime": start_time_new,
                    "EndTime": end_time_new,
                    "ProcessingInterval": pro_interval, 
                    "ReadValueIds": ids
                
                })
            body_list.append(body)
        #
        endpoint = 'values/historicalaggregated'
        # Chunk body
        one_time_body_count = max_workers
        body_chunks_list = list(self.chunk_ids(body_list, one_time_body_count))
        for j,body_chunks in enumerate(body_chunks_list):
        # Request chunkwise data
            results = await asyncio.gather(*[self.http_get_with_aiohttp(session, endpoint,body,timeout) for body in body_chunks])
            print("Success: "+str(j))
            #with open('data/data_chunk_'+str(j)+'.json', 'w') as f:
            #    json.dump(results, f)
            # Create a dataframe and save as parquet
            results_list = []
            for res in results:
                for x in res['HistoryReadResults']:
                    results_list.append(x)
            df_results = pd.DataFrame(results_list)
            df_results.to_parquet('data/data_chunk_'+str(j)+'.parquet')

def get_agg_hist_values(self, server_url: str, start_time: str, end_time: str, pro_interval: int, agg_name: str, node_ids: List[str], include_variables: List):
        """Function to get historical aggregated time value json data for the selected site

        Args:
            server_url (str): server connection url
            start_time (str): start time of requested data
            end_time (str): end time of the requested data
            pro_interval (int): interval time of processing in milliseconds
            node_ids (List[Dict]): node id(s)
            agg_name (str): Name of aggregation
        """
        var_node_ids = [x for x in node_ids if (x.split(".")[-1]) in include_variables]
        read_value_id_dict = [self.create_readvalueids_dict(x,agg_name) for x in var_node_ids]
        body = json.dumps({
                "Connection": {
                    "Url": server_url,
                    "AuthenticationType": 1
                },
                "StartTime": start_time,
                "EndTime": end_time,
                "ProcessingInterval": pro_interval, 
                "ReadValueIds": read_value_id_dict
            
            })
        headers = {'Content-Type': 'application/json'}
        return self.request('POST', 'values/historicalaggregated', body, headers)
    
    def get_agg_hist_values_dataframe(self, server_url: str, start_time: str, end_time: str, pro_interval: int, agg_name: str, data_frame : pd.DataFrame) -> pd.DataFrame:
        """Make a dataframe of aggregated historical value data

        Args:
            server_url (str): server connection url
            start_time (str): start time of requested data
            end_time (str): end time of the requested data
            pro_interval (int): interval time of processing in milliseconds
            agg_name (str): Name of aggregation
            data_frame (pd.DataFrame): Pandas data frame with required columns such as node_ids

        Returns:
            pd.DataFrame: Dataframe of the requested historical values
        """
        # JSON normalization of aggregate historical values
        df = pd.json_normalize(self.get_agg_hist_value_chunks(server_url, start_time, end_time, pro_interval, agg_name, data_frame['VariableId']))
        # Concating aggregated historical values to dataframe
        data_frame1 = pd.concat([data_frame,df], axis=1).drop(columns=['NodeId.IdType', 'NodeId.Id', 'NodeId.Namespace', 'StatusCode.Code', 'StatusCode.Symbol']) 
        # Exploding DataValues column
        df1 = data_frame1.explode('DataValues').reset_index(drop=True)
        # JSON normalization of DataValues
        df2 = pd.json_normalize(df1['DataValues'])
        # Concatenating dataframes
        data_frame2 = pd.concat([df1,df2], axis=1).drop(columns=["DataValues"])
        return data_frame2

    def get_agg_hist_value_chunks(self, server_url: str, start_time: str, end_time: str, pro_interval: int, agg_name: str, node_ids: List[str], include_variables: List, chunk_size=100000,batch_size=1000, max_workers=2):
        """Function to get historical aggregated time value data for a certain time of period

        Args:
            server_url (str): server connection url
            start_time (str): start time of requested data
            end_time (str): end time of the requested data
            pro_interval (int): interval time of processing in milliseconds
            node_ids (List[Dict]): node id(s)
            agg_name (str): Name of aggregation
            chunk_size (int): size of each chunk, default : 100000
            batch_size (int): size of each Id chunck, default : 1000
            max_workers (int): maximum number of workers(CPU), default : 2
        """
        var_node_ids = [x for x in node_ids if (x.split(".")[-1]) in include_variables]
        read_value_ids = [self.create_readvalueids_dict(x,agg_name) for x in var_node_ids]
        # Lenght of time series
        n_datapoints = (pd.to_datetime(end_time) - pd.to_datetime(start_time)).total_seconds()*1000/pro_interval
        # Number of required splits 
        n_time_splits = int(np.ceil(n_datapoints/chunk_size))
        # Ids chunks
        id_chunk_list = list(self.chunk_ids(read_value_ids, batch_size))
        # Get datetime chunks
        start_end_list = self.chunk_datetimes(start_time,end_time, n_time_splits)
        body_elements = list(itertools.product(start_end_list,id_chunk_list))
        # Create body chunks
        body_list = []
        for x in body_elements:
            start_time_new = x[0][0]
            end_time_new = x[0][1]
            ids = x[1]
            body = json.dumps({
                    "Connection": {
                        "Url": server_url,
                        "AuthenticationType": 1
                    },
                    "StartTime": start_time_new,
                    "EndTime": end_time_new,
                    "ProcessingInterval": pro_interval, 
                    "ReadValueIds": ids
                
                })
            body_list.append(body)
        # Chunk body
        one_time_body_count = max_workers * 4
        body_chunks_list = list(self.chunk_ids(body_list, one_time_body_count))
        for j,body_chunks in enumerate(body_chunks_list):
        # Request chunkwise data
            data_list = []
            with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
                future_data_map = {executor.submit(self.request_historical_data, chunk):i for i,chunk in enumerate(body_chunks)}
                for i,future_data in enumerate(concurrent.futures.as_completed(future_data_map)):
                    print("Success: "+str(j)+"_"+str(i))
                    recieved_frame = future_data.result()
                    #with open('data/data_chunk_'+str(j)+"_"+str(i)+'.json', 'w') as f:
                        #json.dump(recieved_frame, f)
                    data_list+=recieved_frame['HistoryReadResults']
            return data_list

def get_agg_hist_value_chunks(self, server_url: str, start_time: str, end_time: str, pro_interval: int, agg_name: str, node_ids: List[str], include_variables: List, chunk_size=100000,batch_size=1000, max_workers=2):
        """Function to get historical aggregated time value data for a certain time of period

        Args:
            server_url (str): server connection url
            start_time (str): start time of requested data
            end_time (str): end time of the requested data
            pro_interval (int): interval time of processing in milliseconds
            node_ids (List[Dict]): node id(s)
            agg_name (str): Name of aggregation
            chunk_size (int): size of each chunk, default : 100000
            batch_size (int): size of each Id chunck, default : 1000
            max_workers (int): maximum number of workers(CPU), default : 2
        """
        var_node_ids = [x for x in node_ids if (x.split(".")[-1]) in include_variables]
        read_value_ids = [self.create_readvalueids_dict(x,agg_name) for x in var_node_ids]
        # Lenght of time series
        n_datapoints = (pd.to_datetime(end_time) - pd.to_datetime(start_time)).total_seconds()*1000/pro_interval
        # Number of required splits 
        n_time_splits = int(np.ceil(n_datapoints/chunk_size))
        # Ids chunks
        id_chunk_list = list(self.chunk_ids(read_value_ids, batch_size))
        # Get datetime chunks
        start_end_list = self.chunk_datetimes(start_time,end_time, n_time_splits)
        body_elements = list(itertools.product(start_end_list,id_chunk_list))
        # Create body chunks
        body_list = []
        for x in body_elements:
            start_time_new = x[0][0]
            end_time_new = x[0][1]
            ids = x[1]
            body = json.dumps({
                    "Connection": {
                        "Url": server_url,
                        "AuthenticationType": 1
                    },
                    "StartTime": start_time_new,
                    "EndTime": end_time_new,
                    "ProcessingInterval": pro_interval, 
                    "ReadValueIds": ids
                
                })
            body_list.append(body)
        # Chunk body
        one_time_body_count = max_workers * 4
        body_chunks_list = list(self.chunk_ids(body_list, one_time_body_count))
        for j,body_chunks in enumerate(body_chunks_list):
        # Request chunkwise data
            data_list = []
            with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
                future_data_map = {executor.submit(self.request_historical_data, chunk):i for i,chunk in enumerate(body_chunks)}
                for i,future_data in enumerate(concurrent.futures.as_completed(future_data_map)):
                    print("Success: "+str(j)+"_"+str(i))
                    recieved_frame = future_data.result()
                    #with open('data/data_chunk_'+str(j)+"_"+str(i)+'.json', 'w') as f:
                        #json.dump(recieved_frame, f)
                    data_list+=recieved_frame['HistoryReadResults']
            return data_list

def read_var_ids(self, json_data : List[Dict], vars:list):
        """Function to select particular variables' node ids

        Args:
            json_data (List[Dict]): API request JSON response 
            vars (list): List of variable(s)
        """
        json_df_merged = self.expand_props_vars(json_data)
        dataframe = json_df_merged[json_df_merged['Variable'].isin(vars)]#['VariableId']
        return dataframe


def request_historical_data(self,body):
        """Request function for http based API request

        Args:
            body (_type_): _description_

        Returns:
            _type_: _description_
        """
        headers = {'Content-Type': 'application/json'}
        #try:
        resp = self.request('POST', 'values/historicalaggregated', body, headers)
        # except:
        #     resp = []
        #     print("Error while requesting historical aggregated data")
        return resp

def get_agg_hist_values(self, server_url: str, start_time: str, end_time: str, pro_interval: int, agg_name: str, node_ids: List[str], include_variables: List):
        """Function to get historical aggregated time value json data for the selected site

        Args:
            server_url (str): server connection url
            start_time (str): start time of requested data
            end_time (str): end time of the requested data
            pro_interval (int): interval time of processing in milliseconds
            node_ids (List[Dict]): node id(s)
            agg_name (str): Name of aggregation
        """
        var_node_ids = [x for x in node_ids if (x.split(".")[-1]) in include_variables]
        read_value_id_dict = [self.create_readvalueids_dict(x,agg_name) for x in var_node_ids]
        body = json.dumps({
                "Connection": {
                    "Url": server_url,
                    "AuthenticationType": 1
                },
                "StartTime": start_time,
                "EndTime": end_time,
                "ProcessingInterval": pro_interval, 
                "ReadValueIds": read_value_id_dict
            
            })
        headers = {'Content-Type': 'application/json'}
        return self.request('POST', 'values/historicalaggregated', body, headers)
    
    def get_agg_hist_values_dataframe(self, server_url: str, start_time: str, end_time: str, pro_interval: int, agg_name: str, include_variables: List, data_frame : pd.DataFrame) -> pd.DataFrame:
        """Make a dataframe of aggregated historical value data

        Args:
            server_url (str): server connection url
            start_time (str): start time of requested data
            end_time (str): end time of the requested data
            pro_interval (int): interval time of processing in milliseconds
            agg_name (str): Name of aggregation
            data_frame (pd.DataFrame): Pandas data frame with required columns such as node_ids

        Returns:
            pd.DataFrame: Dataframe of the requested historical values
        """
        # JSON normalization of aggregate historical values
        df = pd.json_normalize(self.get_agg_hist_values(server_url, start_time, end_time, pro_interval, include_variables, agg_name, data_frame['VariableId']))
        # Concating aggregated historical values to dataframe
        data_frame1 = pd.concat([data_frame,df], axis=1).drop(columns=['NodeId.IdType', 'NodeId.Id', 'NodeId.Namespace', 'StatusCode.Code', 'StatusCode.Symbol']) 
        # Exploding DataValues column
        df1 = data_frame1.explode('DataValues').reset_index(drop=True)
        # JSON normalization of DataValues
        df2 = pd.json_normalize(df1['DataValues'])
        # Concatenating dataframes
        data_frame2 = pd.concat([df1,df2], axis=1).drop(columns=["DataValues"])
        return data_frame2

def json_data_into_dataframe(df_result):
    df_result['Variable'] = df_result['NodeId'].str.split('.').str[-1]
    df = df_result.explode('DataValues').reset_index(drop=True)
    df[['SourceTimestamp','StatusCode', 'Value']] = df['DataValues'].apply(pd.Series)
    df[['Code', 'Status']] = df['StatusCode'].apply(pd.Series)
    df = df.drop(columns=['DataValues','StatusCode'])
    return df

   def get_object_descendants(self, type_name: str, object_Ids: str, domain: str):
        """A function to get object descendants

        Args:
            type_name (str): type_name of a descendant
            object_Ids (List[str]): object id(s) of parent
            domain (str): PV_Assets or PV_Serves

        Returns:
            json data: descendats data of selected object id(s)
        """
        object_type_id = self.get_object_typeid(type_name)
        body = json.dumps({
            "typeId": object_type_id,
            "objectIds": object_Ids,
            "domain": domain
            })
        return self.request('POST', 'query/object-descendants', body)